In [1]:
# main import
import sys
import os
import json
import nltk
import numpy as np
import pandas as pd

# analytics import
sys.path.append('..')
from preprocess import MediumBlogPost

# utils import
from utils import parse_date, get_word_count

# sklearn
from sklearn.model_selection import train_test_split

# visualization imports
import seaborn.apionly as sns
import matplotlib.pyplot as plt
%matplotlib inline


files = ['data-science.jl', 'startup.jl']

ImportError: attempted relative import with no known parent package

## load data

In [16]:
dfs = []
for file_name in files:
    file_path = os.path.join(os.environ['RAW_DATA_PATH'], file_name)
    with open(file_path) as f:
        for line in f:
            # get raw data
            row = MediumBlogPost(**json.loads(line)).to_frame()
            dfs.append(row)

df = pd.concat(dfs).reset_index(drop=True)

In [17]:
df.tail(2)

,title,publish_time,author,url,author_url,headings,contents,mins_read,claps,lang,tags
25019,13 Self-Destructive Habits of Unhappy and Unsu...,2018-08-24,Larry Kim,https://medium.com/marketing-and-entrepreneurs...,https://medium.com/@larrykim,13 Self-Destructive Habits of Unhappy and Unsu...,Stop sabotaging your own best efforts with neg...,6,208,en,"[Self Improvement, Startup, Marketing, Life Le..."
25020,"Slack just raised money at $ 7 B valuation, bu...",2018-08-24,ashu garg,https://medium.com/@ashugarg/slack-just-raised...,https://medium.com/@ashugarg,,Slack recently raised $ 427 M at a valuation o...,3,13,en,"[Slack, Startup, Microsoft, Google, Unicorns]"


## train / test split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('claps', axis=1),
    df.claps,
    test_size=0.2,
    random_state=99)

print(X_train.shape, X_test.shape)

(6901, 10) (1726, 10)


## feature engineer

Base on previous EDA, we think the following features are sensible for predicting claps
1. year, month, day
2. title word count, stop word count, unusual word count
3. headings word count, stop word count, unusual word count
4. contents sentence count, contents word count, stop word count, unusual word count
5. author number of blogs, average claps

In [104]:
get_word_count(df.headings.iloc[:3])

,headings_word_count,headings_stopword_count,headings_unusual_word_count,headings_total_word_count
0,3,2,0,5
1,4,2,4,10
2,21,7,6,34


In [68]:
doc = nlp(df.iloc[250].contents)